In [1]:
from blindNegex import *
import json
import os
from time import time

ROOT_DIR = os.path.dirname(os.path.abspath(""))
negations_file = '../data/negation_speculation_word.txt'
output_file_training = "../data/output_blind_negex_training.json"
output_file_testing = "../data/output_blind_negex_testing.json"

# Load training data and process
data_training = json.load(open(os.path.join(ROOT_DIR, "data", "training_data.json")))
negations = read_negations(negations_file)
process_text(data_training, negations, output_file_training)

# Load testing data and process
data_testing = json.load(open(os.path.join(ROOT_DIR, "data", "test_data.json")))
start_time = time()
process_text(data_testing, negations, output_file_testing)
time_taken = time() - start_time

100%|██████████| 64/64 [00:00<00:00, 80.63it/s]


In [2]:
time_taken

0.8621091842651367